# A. Предсказание движения беспилотного автомобиля

Когда в XIX веке на улицах Великобритании появились первые самоходные повозки, они вызвали у людей скорее страх и недоверие, чем восторг. Поэтому в 1865 году в Великобритании был принят The Locomotive Act, более известный как Red Flag Act, который требовал, чтобы перед каждым автомобилем шёл человек с красным флажком или фонарём. Этот «предвестник прогресса» должен был предупреждать пешеходов и конные экипажи о приближении нового механического транспорта.

Кроме того, закон строго ограничивал скорость автомобилей: не более 2 миль в час в городах и 4 миль в час за их пределами. Эти меры были направлены на то, чтобы адаптировать общество к новым транспортным средствам и минимизировать их риски для безопасности. К концу XIX века стало очевидно, что подобные ограничения только сдерживают прогресс, и в 1896 году Red Flag Act был отменён, а автомобили получили право двигаться быстрее и без «предвестника», предсказывающего появление автомобиля.

Сегодня предсказание маршрута автомобиля стало делом не человека с флажком, а искусственного интеллекта. ИИ способен опираться на огромное количество данных — от состояния дорог и трафика до погодных условий и угла поворота колёс — чтобы не просто направить автомобиль, а выбрать для него наилучший маршрут.

Ваша задача — обучить модель, позволяющую точно моделировать траекторию движения автомобиля на основе поступающих команд управления, технических характеристик и исторических данных о прошлых проездах транспорта по различным дорогам.

## Данные для обучения
Архив YaCupTrain.tar содержит набор из N train записанных сцен проезда легкового автомобиля, разложенных по отдельным папкам. Каждая папка содержит 3 файла:

- metadata.json: содержит общую информацию про сцену
- ride_date — дата проезда
- vehicle_id — уникальный идентификатор автомобиля
- vehicle_model — идентификатор модели автомобиля
- vehicle_model_modification — идентификатор модификации указанной модели автомобиля
- tires — идентификатор типа шин, используемых для колёс передней (front) и задней (rear) оси автомобиля
- location_reference_point_id — идентификатор референсной точки, используемой в качестве начала отсчёта координат в файле localization.csv
- localization.csv: описывает траекторию движения автомобиля на данной 60-секундной сцене. Представляет собой csv файл, каждая строчка которого имеет формат
stamp_ns — время в наносекундах от начала сцены
x, y, z — координаты центра задней оси автомобиля. Считаются в метрах от указанной референсной точки сцены. Направления осей относительно референсной точки: 
x - на восток, 
y - на север, 
z - в небо
roll, pitch, yaw — углы Эйлера в радианах, описывающие ориентацию автомобиля в пространстве. Угол yaw считается относительно оси 
x в направлении оси y.
- control.csv: описывает последовательность команд управления, отправленных автомобилю на протяжении данной сцены.
- stamp_ns — время в наносекундах от начала сцены
- acceleration_level — желаемая интенсивность ускорения. Положительные значения соответствуют силе нажатия на педаль газа, отрицательные — силе нажатия на педаль тормоза
- steering — желаемый угол поворота руля в градусах относительно центрального положения
Обратите внимание, что диапазон значений acceleration_level зависит от модели автомобиля. Также, важно отметить, что данные команды описывают желаемое целевое состояние элементов управления в указанный момент времени, и не обязательно исполняются мгновенно.

## Данные для тестирования
Архив YaCupTest.tar содержит набор из N test    сцен, для которых требуется предсказать новую траекторию автомобиля на основе начального состояния и поступающих команд управления. Каждая папка с тестовым сценарием содержит 4 файла:

- metadata.json: содержит общую информацию про сцену аналогично обучающим данным
- localization.csv: описывает траекторию движения автомобиля в течении первых 5 секунд сцены. Формат аналогичен обучающим данным.
- control.csv: описывает последовательность команд управления в течении первых 20 секунд сцены. Формат аналогичен обучающим данным.
- requested_stamps.csv: содержит одну колонку stamp_ns, содержащую список из T n  моментов времени от начала сцены (в наносекундах) в интервале с 5 по 20 секунду, для которых требуется предсказать положение автомобиля.

## Формат вывода
В качестве решения вам необходимо отправить один файл в формате *.csv, содержащий следующие 5 колонок:

- testcase_id — номер сцены из тестового набора (имя папки от 0 до N test −1)
- stamp_ns — моменты времени из соответствующего файла requested_stamps.csv тестовой сцены.
- x, y, yaw — 3 колонки с предсказанными координатами положения машины и её ориентации на плоскости в указанные моменты времени (В формате аналогичном входным данным).
Таким образом, общее количество строк с предсказаниями в файле с ответом должно совпадать с суммарным количеством таймстемпов в файлах requested_stamps.csv.

- x, y, yaw target

## Calculate metric

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.


## Import libraries

In [73]:
import pandas as pd
import json
import clickhouse_connect
import os
import missingno as msno
from datetime import datetime as dt

from dotenv import load_dotenv
load_dotenv()

# constants
CH_USER = os.getenv("CH_USER")
CH_PASS = os.getenv("CH_PASS")
CH_IP = os.getenv('CH_IP')

# from tools import create_db_table_from_df, pd_tools, spark_tools

root_path = "."
tmp_path = f'{root_path}/tmp'
data_path = f'{root_path}/data/self-drive'
data_train_path = f'{data_path}/train_data'
data_test_path = f'{data_path}/test_data'
tmp_data_path=f'{data_path}/tmp_data'

ch_client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CH_USER, password=CH_PASS)

## Data load

In [2]:
# read folder names in path
def read_names(path: str):
    '''Read folder names or file names in the path'''
    return os.listdir(path)

train_ids = pd.Series(read_names(data_train_path)).apply(int)
test_ids = pd.Series(read_names(data_test_path)).apply(int)
train_ids = train_ids.sort_values().reset_index(drop=True)
train_ids



0            0
1            1
2            2
3            3
4            4
         ...  
41995    41995
41996    41996
41997    41997
41998    41998
41999    41999
Length: 42000, dtype: int64

In [3]:
files_temp = read_names(f'{data_train_path}/{train_ids[0]}')
files_temp

['control.csv', 'localization.csv', 'metadata.json']

In [27]:
control_temp = pd.read_csv(f'{data_train_path}/{train_ids[0]}/{files_temp[0]}')
control_temp.head()

,stamp_ns,acceleration_level,steering
0,2987440736,-114,-2.655140
1,3027341070,-123,-2.598169
2,3066793076,-132,-2.544422
3,3106757146,-141,-2.544422
4,3146784622,-147,-2.488557


In [28]:
localization_temp = pd.read_csv(f'{data_train_path}/{train_ids[0]}/{files_temp[1]}')
localization_temp.head()

,stamp_ns,x,y,z,roll,pitch,yaw
0,0,-4292.313705,-14527.266319,66.043314,0.003926,-0.054198,-1.936810
1,39989868,-4292.489928,-14527.726083,66.070022,0.003702,-0.054172,-1.936858
2,79819886,-4292.662729,-14528.183063,66.090338,0.002404,-0.054628,-1.936827
3,125154671,-4292.862032,-14528.702952,66.120814,0.002709,-0.054559,-1.936894
4,159636974,-4293.011898,-14529.097871,66.138226,0.003264,-0.053668,-1.936876


In [22]:
def interpolate_coords_eyler(control:pd.DataFrame, localization:pd.DataFrame):

    def find_min_max(control:pd.DataFrame, localization:pd.DataFrame):
        '''Find min and max timestamp in localization for each timestamp in control dataframe'''
        control['loc_stamp_max'] = control['stamp_ns'].apply(lambda x: localization[localization['stamp_ns'] >= x]['stamp_ns'].min())
        control['loc_stamp_min'] = control['stamp_ns'].apply(lambda x: localization[localization['stamp_ns'] < x]['stamp_ns'].max())
        control_2m = control.copy()
        return control_2m

    def merge_min_max(control_2m:pd.DataFrame, localization:pd.DataFrame):
        '''Merge min and max timestamp in localization for each timestamp in control dataframe'''
        control_3m = (control_2m.merge(localization, left_on='loc_stamp_max', right_on='stamp_ns', how='left', suffixes=('', '_max'))
                    .merge(localization, left_on='loc_stamp_min', right_on='stamp_ns', how='left', suffixes=('', '_min'))
        )
        control_3m.rename(columns={'x':'x_max', 'y':'y_max', 'z':'z_max', 'roll':'roll_max', 'pitch':'pitch_max', 'yaw':'yaw_max'}, inplace=True)
        control_3m.drop(columns=['loc_stamp_max', 'loc_stamp_min'], inplace=True)
        return control_3m

    def interpolate_coords(control_3m, col_min:str, col_max:str):
        '''Interpolate values between max and min values'''
        control_inter = (control_3m[['stamp_ns', 'stamp_ns_max', 'stamp_ns_min', col_min, col_max]]
                    .apply(lambda x: (x['stamp_ns'] - x['stamp_ns_min']) / (x['stamp_ns_max'] - x['stamp_ns_min']) * (x[col_max] - x[col_min]) + x[col_min], axis=1)
                    )
        return control_inter

    control_2m = find_min_max(control, localization)
    control_3m = merge_min_max(control_2m, localization)

    coords_cols = ['x', 'y', 'z', 'roll', 'pitch', 'yaw']
    contr_cols = ['stamp_ns', 'acceleration_level', 'steering']

    for col in coords_cols:
        control_3m[col] = interpolate_coords(control_3m, f'{col}_min', f'{col}_max')

    control_inter = control_3m[contr_cols + coords_cols]

    return control_inter




In [23]:
interpolate_coords_eyler(control_temp, localization_temp)

,stamp_ns,acceleration_level,steering,x,y,z,roll,pitch,yaw
0,2987440736,-114,-2.655140,-4305.325027,-14560.800637,67.786293,0.002836,-0.048921,-1.945764
1,3027341070,-123,-2.598169,-4305.489155,-14561.217631,67.808224,0.002993,-0.049162,-1.945839
2,3066793076,-132,-2.544422,-4305.652097,-14561.630123,67.833736,0.005068,-0.049696,-1.945933
3,3106757146,-141,-2.544422,-4305.815555,-14562.044470,67.857731,0.006305,-0.050110,-1.946037
4,3146784622,-147,-2.488557,-4305.979063,-14562.457108,67.880212,0.007713,-0.049996,-1.946176
...,...,...,...,...,...,...,...,...,...
1495,62786741116,33,117.135357,-4445.835750,-14933.380856,66.769112,0.005987,-0.001311,-1.681841
1496,62826899778,33,119.059706,-4445.859184,-14933.601311,66.769470,0.006965,-0.001850,-1.671587
1497,62867315073,33,120.952111,-4445.882071,-14933.823765,66.767511,0.008721,-0.002507,-1.661061
1498,62906605994,32,122.802597,-4445.901307,-14934.040765,66.766964,0.009677,-0.002989,-1.650662


In [84]:
metadata_temp = pd.read_json(f'{data_train_path}/{train_ids[0]}/{files_temp[2]}')
metadata_temp

,ride_date,tires,vehicle_id,vehicle_model,vehicle_model_modification,location_reference_point_id
front,2022-03-14,0,0,0,0,0
rear,2022-03-14,0,0,0,0,0


In [85]:
# change tires column to front and rear columns
def tires_to_columns_date(metadata:pd.DataFrame):
    '''Change tires column to front and rear columns and 
        and convert ride_date to datetime and add year, month, day columns'''
    metadata['front_tire'] = metadata['tires'][0]
    metadata['rear_tire'] = metadata['tires'][1]
    metadata = metadata.drop(columns=['tires']).reset_index(drop=True).loc[:0]
    # convert ride_date to datetime and add year, month, day columns
    metadata['ride_date'] = pd.to_datetime(metadata['ride_date'])
    metadata['ride_year'] = metadata['ride_date'].dt.year
    metadata['ride_month'] = metadata['ride_date'].dt.month
    metadata['ride_day'] = metadata['ride_date'].dt.day
    metadata = metadata.drop(columns=['ride_date'])
    return metadata


In [86]:
metadata_temp = tires_to_columns_date(metadata_temp)
metadata_temp

,vehicle_id,vehicle_model,vehicle_model_modification,location_reference_point_id,front_tire,rear_tire,ride_year,ride_month,ride_day
0,0,0,0,0,0,0,2022,3,14


In [49]:
metadata_temp['ride_date'][0].split('-')[0]

'2022'

In [45]:
def date_to_nums(metadata:pd.DataFrame):
    '''Change date columns to numbers'''
    metadata['date'].split('-')
    return metadata

ride_date                      object
tires                           int64
vehicle_id                      int64
vehicle_model                   int64
vehicle_model_modification      int64
location_reference_point_id     int64
front_tire                      int64
rear_tire                       int64
dtype: object

In [89]:
# duplicate metadata rows by the number of localization rows
metadata_temp = metadata_temp.loc[metadata_temp.index.repeat(len(localization_temp))].reset_index(drop=True)
metadata_temp.head()

,ride_date,tires,vehicle_id,vehicle_model,vehicle_model_modification,location_reference_point_id
0,2022-03-14,0,0,0,0,0
1,2022-03-14,0,0,0,0,0
2,2022-03-14,0,0,0,0,0
3,2022-03-14,0,0,0,0,0
4,2022-03-14,0,0,0,0,0


In [93]:
# add metadata to localization
loc_met = localization_temp.merge(metadata_temp, left_index=True, right_index=True)
loc_met.head()

,stamp_ns,x,y,z,roll,pitch,yaw,ride_date,tires,vehicle_id,vehicle_model,vehicle_model_modification,location_reference_point_id
0,0,-4292.313705,-14527.266319,66.043314,0.003926,-0.054198,-1.936810,2022-03-14,0,0,0,0,0
1,39989868,-4292.489928,-14527.726083,66.070022,0.003702,-0.054172,-1.936858,2022-03-14,0,0,0,0,0
2,79819886,-4292.662729,-14528.183063,66.090338,0.002404,-0.054628,-1.936827,2022-03-14,0,0,0,0,0
3,125154671,-4292.862032,-14528.702952,66.120814,0.002709,-0.054559,-1.936894,2022-03-14,0,0,0,0,0
4,159636974,-4293.011898,-14529.097871,66.138226,0.003264,-0.053668,-1.936876,2022-03-14,0,0,0,0,0


In [91]:
loc_met.shape

(1582, 13)

In [92]:
loc_met_ctrl = loc_met.copy()

In [76]:
# think about stapm_lc before or after stamp_ct

for i, stamp_lc in enumerate(loc_met_ctrl['stamp_ns']):
    for n, stamp_ct in enumerate(control_temp['stamp_ns']):
        if  n < len(control_temp['stamp_ns'])-1 and stamp_lc > stamp_ct and stamp_lc < control_temp['stamp_ns'][n+1]:
            # print(i, n)
            loc_met_ctrl.at[i, 'stamp_ns_control'] = control_temp['stamp_ns'][n]
            loc_met_ctrl.at[i, 'acceleration_level'] = control_temp['acceleration_level'][n]
            loc_met_ctrl.at[i, 'steering'] = control_temp['steering'][n]
            break
  
loc_met_ctrl.fillna(0, inplace=True)
loc_met_ctrl   


,stamp_ns,x,y,z,roll,pitch,yaw,ride_date,tires,vehicle_id,vehicle_model,vehicle_model_modification,location_reference_point_id,stamp_ns_control,acceleration_level,steering
0,0,-4292.313705,-14527.266319,66.043314,0.003926,-0.054198,-1.936810,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
1,39989868,-4292.489928,-14527.726083,66.070022,0.003702,-0.054172,-1.936858,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
2,79819886,-4292.662729,-14528.183063,66.090338,0.002404,-0.054628,-1.936827,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
3,125154671,-4292.862032,-14528.702952,66.120814,0.002709,-0.054559,-1.936894,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
4,159636974,-4293.011898,-14529.097871,66.138226,0.003264,-0.053668,-1.936876,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577,63079991016,-4445.953846,-14935.006202,66.762048,0.002899,-0.001396,-1.602179,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
1578,63120445325,-4445.959637,-14935.233032,66.762420,0.001984,-0.001401,-1.590317,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
1579,63160340945,-4445.966875,-14935.455432,66.758763,-0.000359,-0.001176,-1.578275,2022-03-14,0,0,0,0,0,0.0,0.0,0.0
1580,63200463435,-4445.967213,-14935.680835,66.758973,0.000547,-0.000704,-1.565992,2022-03-14,0,0,0,0,0,0.0,0.0,0.0


In [77]:
loc_met_ctrl['acceleration_level'].value_counts()

acceleration_level
 0.0      95
-43.0     32
 43.0     21
-39.0     17
 20.0     16
          ..
-211.0     1
 93.0      1
 149.0     1
-192.0     1
-203.0     1
Name: count, Length: 317, dtype: int64

In [ ]:
# read data in each file
def read_data(path: str, ids: pd.Series, files: list):
    '''Read data in each file'''
    for i in ids:
        data = []
        for file in files:
            if file.endswith('.csv'):
                data.append(pd.read_csv(f'{path}/{i}/{file}'))
            elif file.endswith('.json'):
                data.append(json.load(open(f'{path}/{i}/{file}')))
    return data

In [7]:
data_train_path

'./data/train_data'

In [6]:
# Load all ids of a dataset

def read_testcase_ids(dataset_path: str):
    ids = [int(case_id) for case_id in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, case_id))]
    return ids

train_ids = read_testcase_ids(data_train_path)
test_ids = read_testcase_ids(data_test_path)

train_ids.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/train_data'